# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

In [9]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## OVERVIEW

In [12]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [13]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

## SUMMARY STATISTICS
Let's start by calculating some summary statistics and look for any outlier countries.

In [15]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

## Exploring Outliers

In [18]:
%%sql
SELECT * FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [20]:
%%sql
SELECT * FROM facts
WHERE population == (SElECT MAX(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the [CIA Factbook page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):



Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

In [22]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

## Exploring Average Population and Area¶
Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [25]:
%%sql
SELECT AVG(population)as avg_population, AVG(area) AS avg_area
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

## Finding Densely Populated Countries¶
To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

Above average values for population.
Below average values for area.

In [27]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


What country has the most people? What country has the highest growth rate?

In [33]:
%%sql 
SELECT name, ROUND(birth_rate - death_rate, 2) AS growth_rate
  FROM facts
 WHERE growth_rate IS NOT NULL
 ORDER BY growth_rate
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,growth_rate
Bulgaria,-5.52
Serbia,-4.58
Latvia,-4.31
Lithuania,-4.17
Ukraine,-3.74
Hungary,-3.57
Germany,-2.95
Slovenia,-2.95
Romania,-2.76
Croatia,-2.73


 Which countries have more water than land?
Which countries will add the most people to their population next year?
Which countries have a higher death rate than birth rate?
What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

In [32]:
%%sql
SELECT name AS Country, population AS Population
FROM facts
WHERE Country <> 'World'
ORDER BY population DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


Country,Population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


Which countries have the highest ratios of water to land?

In [35]:
%%sql
SELECT name, area_land, area_water,
    ROUND(CAST(area_water AS FLOAT) / area_land, 2) AS water_to_land
  FROM facts
 ORDER BY water_to_land DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area_land,area_water,water_to_land
British Indian Ocean Territory,60,54340,905.67
Virgin Islands,346,1564,4.52
Puerto Rico,8870,4921,0.55
"Bahamas, The",10010,3870,0.39
Guinea-Bissau,28120,8005,0.28
Malawi,94080,24404,0.26
Netherlands,33893,7650,0.23
Uganda,197100,43938,0.22
Eritrea,101000,16600,0.16
Liberia,96320,15049,0.16
